<a href="https://colab.research.google.com/github/daripp/root_area_counter/blob/main/New_Root_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')
!ls 

Mounted at /content/drive
drive  sample_data


In [ ]:
#########!!!!!!!!!!!!!!!!!!!!!CODE For ROOTS!!!!!!!!!!!!!!!!!!

import glob
import numpy as np
import matplotlib.pyplot as plt
import skimage as skimage
import scipy as scipy
import cv2 as cv2
import pandas as pd
import os
from skimage import filters, io, img_as_ubyte
from skimage.color import rgb2grey, label2rgb
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
from skimage.exposure import match_histograms
from skimage.filters import sobel, laplace, gaussian
from skimage import feature
from scipy.ndimage import morphology
from skimage.morphology import disk, binary_dilation, diamond
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
#*********************************************Put folder name here**************************************************
directn = 'Folder_name_here'
filenames = os.listdir(directn)
for filename in os.listdir(directn):
#*******************************************makes new directory called "(directory name here) + cropped"********************************************
    new_dir_name=directn+'_extension_to_folder_name'
    if not os.path.exists(new_dir_name):
        os.makedirs(new_dir_name)
    #image is read from directory to system  

    
    camera1=io.imread(directn+'/'+filename)
    image = rgb2grey(camera1)
    img=camera1    
    b,g,r = cv2.split(img)        
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Adjust Dilation and remove small objects values here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
    binary_dilation= disk(3)
    remove_small_objects= 2000
    edges_r=np.array(r, dtype='ubyte')
    edges_b=np.array(b, dtype='ubyte')
    edges_g=np.array(g, dtype='ubyte')
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Adjust threshold values here; values are additive!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    edges_r1=np.where((edges_r < 70) & (edges_r > 30), 255, 0)
    edges_b1=np.where((edges_b < 120) & (edges_b > 80), 255, 0)
    edges_g1=np.where((edges_g < 15) & (edges_g > 5), 255, 0)

    edges_1r1=np.where((edges_r < 30) & (edges_r > 70), 255, 0)
#!!!!!!!!!!!Images converted to 0-255 scale!!!!!!!!!!!!!!!!!!!!!!!!    
    edges_r2=img_as_ubyte(edges_r1)
    edges_b2=img_as_ubyte(edges_b1)
    edges_g2=img_as_ubyte(edges_g1)
    # #edges2= feature.canny(edges2, sigma=1.5)
#!!!!!!!!!!!!!!!!!!!!Mask expansion, adjust values above!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
    edges_r2=skimage.morphology.binary_dilation(edges_r2, binary_dilation)
    edges_b2=skimage.morphology.binary_dilation(edges_b2, binary_dilation)
    edges_g2=skimage.morphology.binary_dilation(edges_g2, binary_dilation)
###!!!!!!!!!!!!!!!!!!!!Removes small objects marked by the computer!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
    edges_r3 = skimage.morphology.remove_small_objects(edges_r2, min_size=remove_small_objects)
    edges_b3 = skimage.morphology.remove_small_objects(edges_b2, min_size=remove_small_objects)
    edges_g3 = skimage.morphology.remove_small_objects(edges_g2, min_size=remove_small_objects)
###*************************************************!!!!!!!!!!!!!!!!Combine layers here! You don't have to combine if not helpful, but you have to tell the computer what you want!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    edges_combine=edges_r3+edges_b3
#***************Redefine layers as np arrays for subtraction*************************************************    
    edges_combine=np.array(edges_combine)
    edges_r3=np.array(edges_r3)
    edges_b3=np.array(edges_b3)

    np.subtract(edges_combine, edges_r3, dtype=np.float)
    
    edges_bsubtractr=np.subtract(edges_combine, edges_r3, dtype=np.float)
    edges_rsubtractb=np.subtract(edges_combine, edges_b3, dtype=np.float)
#**!!!!!!!!!!!!!!!Fills in holes after layers are combined!!!!!!!!!!!!!!!!!!!!!    
    #edges_rb3=scipy.ndimage.morphology.binary_fill_holes(edges_combine)
#!!!!!!!!!!!Generates label from combined images, back ground is a little wonky as the output is being inverted by  
    label_image = label(edges_combine, background=0)
    image_label_overlay = label2rgb(label_image, image=camera1, bg_label=255, alpha=0.2)

    d=measure.regionprops_table(label_image, image, properties=['label','area','centroid'])
    data = pd.DataFrame(d)
    # data.sort_values('area', inplace=True, ascending=False)
    # label_images=label_image<1
    # label_image[label_images]=255
  #  print(data)
#edges = clear_border(edges)
#camera=np.array(camera)
#camera=np.array(np.count_nonzero(255))
#print(camera)
#inds_x = np.arange(len(camera))
#inds_y = (4 * inds_x) % len(camera)
#camera[inds_x, inds_y] = 0

#io.imshow(edges)
#io.imshow(edges)
    for j in range(len(d['centroid-0'])):
        print(j)

        position = (d['centroid-1'][j],d['centroid-0'][j])
        cv2.putText(
         image_label_overlay, #numpy array on which text is written
         str(d['label'][j]), #text
         position, #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         5, #font size
         (0, 0, 0, 0),15)

    
      
   
        


    plt.figure(dpi=1000)
    plt.imshow(edges_r3, cmap='binary')
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_red.png')
    plt.close()

    # plt.figure(dpi=300)
    # plt.imshow(edges_g3, cmap='binary')
    # plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_green.tif')
    # plt.close()

    plt.figure(dpi=1000)
    plt.imshow(edges_b3, cmap='binary')
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_blue.png')
    plt.close()

    plt.figure(dpi=1000)
    plt.imshow(edges_combine, cmap='binary')
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_combined.png')
    plt.close()

    plt.figure(dpi=1000)
    plt.imshow(edges_bsubtractr, cmap='binary')
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_b_csubtractr')
    plt.close()

    plt.figure(dpi=1000)
    plt.imshow(edges_rsubtractb, cmap='binary')
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'_r_csubtractb.png')
    plt.close()

    plt.figure(dpi=1000)
    plt.imshow(image_label_overlay)
    plt.savefig(new_dir_name+'/'+filename.split('.')[0]+'combined_image_label_overlay.png')
    data.to_excel(new_dir_name+'/'+filename.split('.')[0]+'combined_image_labeloverlaydata.xlsx')
    plt.close()


    # io.imsave(new_dir_name+'/'+filename.split('.')[0]+'red.jpeg', edges_r3)
    # io.imsave(new_dir_name+'/'+filename.split('.')[0]+'blue.jpeg', edges_b3)
    # io.imsave(new_dir_name+'/'+filename.split('.')[0]+'green.jpeg', edges_g3)

    # data.to_excel(new_dir_name+'/'+filename.split('.')[0]+'.xlsx')